In [3]:
#import numpy as np
import pandas as pd
import missingno as msno
import data_cleaning as dc
import pickle

# Packages pour la visualisation

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pandas.plotting import scatter_matrix
%matplotlib inline

import visu


from importlib import reload

In [4]:
reload(dc);

### Gestion des NaN

In [5]:
# Chargement des donnees mergees
df = dc.load_datasets()
df.shape

(471149, 16)

In [6]:
# Variables numériques
df_num = df.select_dtypes(include = "number")
df_num.describe().T

,count,mean,std,min,25%,50%,75%,max
residueCount,471149.0,6257.931820,2.361838e+04,0.00,456.00,1140.00,4528.00,313236.0
resolution,449486.0,3.020658,3.091223e+00,0.48,2.00,2.50,3.10,70.0
structureMolecularWeight,471149.0,926115.287672,3.018842e+06,453.55,52766.84,131138.55,635323.06,97730536.0
crystallizationTempK,317697.0,290.889746,8.785254e+00,4.00,291.00,293.00,295.00,398.0
densityMatthews,389645.0,2.850779,8.243937e-01,0.00,2.32,2.67,3.19,99.0
densityPercentSol,389768.0,54.198949,1.026988e+01,0.00,46.89,53.95,61.37,92.0
phValue,340694.0,6.830608,2.461778e+00,0.00,6.10,7.00,7.50,724.0
publicationYear,413513.0,2010.483702,6.997897e+00,201.00,2007.00,2012.00,2015.00,2018.0


In [7]:
# Variables catégorielles
df_cat = df.select_dtypes(include = "object")
df_cat.describe().T

,count,unique,top,freq
structureId,471149,140250,3J3Q,1356
classification,471145,4989,RIBOSOME,60710
experimentalTechnique,471149,32,X-RAY DIFFRACTION,405579
macromoleculeType,435379,13,Protein,346325
crystallizationMethod,314754,546,"VAPOR DIFFUSION, HANGING DROP",167555
pdbxDetails,385468,90933,"PEG 20K, KSCN, Mg Acetate, Tris-Acetate, Glyce...",3709
chainId,471139,2836,A,135653
sequence,471121,104812,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...,5112


In [8]:
dc.show_missing(df)

,Missing Values,% Value
crystallizationMethod,156395,33.200000
crystallizationTempK,153452,32.600000
phValue,130455,27.700000
pdbxDetails,85681,18.200000
densityMatthews,81504,17.300000
densityPercentSol,81381,17.300000
publicationYear,57636,12.200000
macromoleculeType,35770,7.600000
resolution,21663,4.600000
sequence,28,0.000000


### Stratégie de gestion des NaN pour les variables numériques :
#### 1. Supprimer la variable [publicationYear'] qui contient les dates de publication, pas d'impact direct sur la classification
#### 2. Remplacer les valeurs manquantes par la valeur médiane 

### Stratégie de gestion des NaN pour les variables catégorielles : 
#### 1. Ne pas considérer les variables [sequence, pdbxDetails, chainId, structureId] car par définition, la séquence n'est pas prise en compte dans l'étude. Les variables [chainId, structureId] sont des ids sans impact dans l'étude. Et pdbxDetails regroupe les caractérisques diverses de la protéine en une chaîne de caractères.
#### 2. Remplacer [macromoleculeType, crystallizationMethod,  classification] par le mode
 

### Gestion du nombre de modalités présentes dans les variables du dataframe : 
#### - Conserver les modalités les plus importantes et regrouper les autres dans une seule modalité :others_xxx 
#### (L'exploration des données a permis d'extraire les modalités importantes, et de comprendre les plus importantes. Supprimer les autres modalités réduit considérablement la taille du dataset, le choix de les regrouper permet de conserver un nombre de données important pour l'analyse)
### Pour la variable classification (variable cible), conserver uniquement celles dont la modalité regroupe plus de 5000 valeurs, et regrouper les autres en une seule

In [9]:
#Toutes ces opérations sont effectuées dans la fonction clean_data du module dc
df = dc.clean_data(df)
df.shape

(471149, 11)

In [14]:
ex = (df.classification.value_counts(ascending = False, normalize = True)*100)
#ex.[:50].plot(kind = 'bar', figsize = (12,10));
visu.plotly_pie(ex,'classification', ex.index, 'Répartition de la variable classification après nettoyage et mise en forme des données')